In [403]:
import pandas as pd
from keras.preprocessing.sequence import TimeseriesGenerator

mods2_model = {
    'cfg': {
        'model_name': 's24-l6-diff',
        'differential': True,
        'forecast_steps': 3,
        'train': {
            'epochs': 100,
            'epochs_incremental': 100
        },
        'tsg': {
            'length': 3,
            'sampling_rate': 1,
            'stride': 1,
            'batch_size': 1
        },
        'data': {
            'split': 0.8,
            'X': [
                'a',
                'b'
            ],
            'Y': [
                'a',
                'b'
            ]
        }        
    }
}

# mods2_model['cfg']['differential'] ==> delta
# mods2_model['cfg']['forecast_steps'] ==> steps_ahead
# mods2_model['cfg']['tsg']['length'] ==> sequence_length

def tsg2tsv(tsg):
    ret = ''
    for i in range(len(tsg)):
        x, y = tsg[i]
        ret += '%s => %s\n' % (x, y)
    return ret

def differentiate(df:pd.DataFrame, k) -> pd.DataFrame:
    return df[k:]-df[:-k].values

def inverse_differentiate(df:pd.DataFrame, seen:pd.DataFrame, k) -> pd.DataFrame:
    return seen.values+df

def transform(mods2_model:dict, df:pd.DataFrame) -> pd.DataFrame:
    isdiff = mods2_model['cfg']['differential']
    if isdiff:
        forecast_steps = mods2_model['cfg']['forecast_steps']
        return differentiate(df, forecast_steps)
    else:
        return df

def inverse_transform(mods2_model:dict, df:pd.DataFrame, df_orig:pd.DataFrame) -> pd.DataFrame:
    isdiff = mods2_model['cfg']['differential']
    if isdiff:
        sequence_length = mods2_model['cfg']['tsg']['length']
        forecast_steps = mods2_model['cfg']['forecast_steps']
        prev = df_orig[-1:].values
        return df + prev
    else:
        return df

def create_tsg(mods2_model, X, Y):
    forecast_steps = mods2_model['cfg']['forecast_steps']
    args = mods2_model['cfg']['tsg']
    if forecast_steps > 1:
        return TimeseriesGenerator(
            X[:-forecast_steps+1],
            Y[forecast_steps-1:],
            **args
        )
    else:
        return TimeseriesGenerator(
            X,
            Y,
            **args
        )

df = pd.DataFrame([[1,2],[11,12],[9,10],[5,6],[3,4],[7,8],[13,14],[5,2],[6,3],[3,1],[2,4]], columns=['a', 'b'])
display(df)

,a,b
0,1,2
1,11,12
2,9,10
3,5,6
4,3,4
5,7,8
6,13,14
7,5,2
8,6,3
9,3,1


In [398]:
df_t = transform(mods2_model, df)
display(df_t)

,a,b
3,4,4
4,-8,-8
5,-2,-2
6,8,8
7,2,-2
8,-1,-5
9,-10,-13
10,-3,2


In [399]:
tsg = create_tsg(mods2_model, df_t.values, df_t.values)
print(tsg2tsv(tsg))

[[[ 4  4]
  [-8 -8]
  [-2 -2]]] => [[-1 -5]]
[[[-8 -8]
  [-2 -2]
  [ 8  8]]] => [[-10 -13]]
[[[-2 -2]
  [ 8  8]
  [ 2 -2]]] => [[-3  2]]



In [400]:
isdiff = mods2_model['cfg']['differential']
sequence_length = mods2_model['cfg']['tsg']['length']
forecast_steps = mods2_model['cfg']['forecast_steps']

step = 1
X = df[step:step+sequence_length+(forecast_steps if isdiff else 0)]
Xt = transform(mods2_model, X)

print('X:')
display(X)

print('Xt:')
display(Xt)

# truth value
Y_true = df[X.index[-1]+forecast_steps:X.index[-1]+forecast_steps+1]

# simulate prediction
pred = tsg[step][1]
print('\npred = model.predict(Xt) = %s' % pred)

# last seen values. predicted delta is added to this value in inverse transform
Y_prev = X
pred_t = inverse_transform(mods2_model, pred, Y_prev)
print('\npred_t = %s' % pred_t)


Y = Y_true.copy(deep=1)
Y[:] = pred_t
print('\nprediction:\n')
display(Y)
print('\ntruth:\n')
display(Y_true)

X:


,a,b
1,11,12
2,9,10
3,5,6
4,3,4
5,7,8
6,13,14


Xt:


,a,b
4,-8,-8
5,-2,-2
6,8,8



pred = model.predict(Xt) = [[-10 -13]]

pred_t = [[3 1]]

prediction:



,a,b
9,3,1



truth:



,a,b
9,3,1
